In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
file_train = '../input/jobathon-may-2021-credit-card-lead-prediction/train.csv'
file_test = '../input/jobathon-may-2021-credit-card-lead-prediction/test.csv'
file_submission = '../input/jobathon-may-2021-credit-card-lead-prediction/sample_submission.csv'

In [ ]:
train = pd.read_csv(file_train)
test = pd.read_csv(file_test)
submission = pd.read_csv(file_submission)

In [ ]:
train['Age'].plot(kind = 'hist')

In [ ]:
train['Avg_Account_Balance'].plot(kind = 'hist',logx = True)

There are 245725 training sets and 105312 test sets

In [ ]:
train.describe()

In [ ]:
train.Avg_Account_Balance.min(),train.Avg_Account_Balance.max()

Only Credit_Product has nan values nad rest all columns have are good

Lets find out if we can find any relationship of Credit_Product with other columns to fill the Nan values

In [ ]:
train.Is_Lead.value_counts().plot(kind = 'bar')

We can see that is imbalanced training set so we have to tackle this problem also.

In [ ]:
train['Credit_Product'].fillna('Yes',inplace= True)
test['Credit_Product'].fillna('Yes',inplace= True)

In [ ]:
train = train.drop(['ID'],axis = 1)
test = test.drop(['ID'],axis = 1)

In [ ]:
labels = train['Is_Lead']
train = train.drop(['Is_Lead'],axis = 1)

In [ ]:
os = RandomOverSampler(sampling_strategy = 1)

x_res,y_res = os.fit_resample(train,labels)

In [ ]:
x_res = pd.DataFrame(x_res,columns = ['Gender','Age',	'Region_Code',	'Occupation','Channel_Code',	'Vintage',	'Credit_Product',	'Avg_Account_Balance',	'Is_Active'])
y_res = pd.DataFrame(y_res,columns = ['labels'])

In [ ]:
train.Credit_Product.value_counts()

In [ ]:
train.isna().sum()

In [ ]:
#Age_cat = pd.cut(x_res.Age,bins=[22,33,60,86],labels=['Low','Mid','High'])
balance = pd.cut(x_res.Avg_Account_Balance,bins=[0,605000,900000,1400000,100000000],labels=[0,1,2,3])

In [ ]:

#x_res.insert(2,'Age Group',Age_cat)
#test.insert(2,'Age Group',Age_cat)
x_res.insert(9,'balance Group',balance)
test.insert(9,'balance Group',balance)

In [ ]:
#x_res['Age Group'] = x_res['Age Group'].astype('object')
#test['Age Group'] = test['Age Group'].astype('object')
x_res['balance Group'] = x_res['balance Group'].astype('object')
test['balance Group'] = test['balance Group'].astype('object')

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
x_res['Gender'] = label_encoder.fit_transform(x_res['Gender'])
x_res['Region_Code'] = label_encoder.fit_transform(x_res['Region_Code'])
x_res['Occupation'] = label_encoder.fit_transform(x_res['Occupation'])
x_res['Channel_Code'] = label_encoder.fit_transform(x_res['Channel_Code'])
x_res['Credit_Product'] = label_encoder.fit_transform(x_res['Credit_Product'])
x_res['Is_Active'] = label_encoder.fit_transform(x_res['Is_Active'])
#x_res['Age Group'] = label_encoder.fit_transform(x_res['Age Group'])
x_res['balance Group'] = label_encoder.fit_transform(x_res['balance Group'])

test['Gender'] = label_encoder.fit_transform(test['Gender'])
test['Region_Code'] = label_encoder.fit_transform(test['Region_Code'])
test['Occupation'] = label_encoder.fit_transform(test['Occupation'])
test['Channel_Code'] = label_encoder.fit_transform(test['Channel_Code'])
test['Credit_Product'] = label_encoder.fit_transform(test['Credit_Product'])
test['Is_Active'] = label_encoder.fit_transform(test['Is_Active'])
#x_res['Age Group'] = label_encoder.fit_transform(x_res['Age Group'])
x_res['balance Group'] = label_encoder.fit_transform(x_res['balance Group'])


In [ ]:
#x_res = x_res.drop(['Age'],axis = 1)
#test = test.drop(['Age'],axis = 1)

x_res = x_res.drop(['Avg_Account_Balance'],axis = 1)
test = test.drop(['Avg_Account_Balance'],axis = 1)

# Creating BaseLine Model using RandomForestClassifier

In [ ]:
x_res.head(10)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_res,y_res,test_size =0.2,random_state = 48)

In [ ]:
clf = RandomForestClassifier(
    n_estimators = 1500,
    max_depth = 7,
    min_samples_leaf = 50,
    n_jobs = -1,
    verbose = True
    )
model = clf.fit(X_train,y_train)

In [ ]:
predict_values = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict_values)

In [ ]:
predict_train = clf.predict(X_train)

In [ ]:
roc_auc_score(y_train,predict_train)

In [ ]:
predict = clf.predict(test)

In [ ]:
submission['Is_Lead'] = predict

In [ ]:
submission.to_csv("submission.csv", index = False)